In [1]:
import numpy as np
import pandas as pd
import os
import openpyxl
from openpyxl.styles import Font, Alignment, PatternFill, Color, Border, Side

import warnings
warnings.filterwarnings('ignore')

## 매뉴얼

1. 경로 설정의 '변경해야 하는 경로'를 설정한다.(처음 1회만 설정하면 됨)
2. 모니터링 월 설정에서 원하는 월을 '숫자'로 넣는다.
3. 메뉴의 Kernel → Restart & Run All 를 클릭 후 실행


## 🔨모니터링 월 설정

In [2]:
# 변경사항(원하는 모니터링 월을 숫자로 적는다.)

monitering_month = 3


## 경로 설정

In [3]:
# 고정 경로
# countrys =['Aruba (AW)']
# as_countrys = ['aw']
# gather_countrys = ['Aruba']

countrys =['Aruba (AW)', 'Australia (AU)', 'Azerbaijan (AZ)', 'British Dominica (DM)', 'Brunei (BN)', 'Chile (CL)',
           'Colombia (CO)', 'Costa Rica (CR)', 'Ecuador (EC)','Fiji (FJ)','Georgia (GE)','Honduras (HN)',
           'Hong Kong (HK)','Indonesia (ID)','Kuwait (KW)','Laos (LA)','Lebanon (LB)','Malaysia (MY)',
           'Mauritius (MU)', "Mexico (MX)","Mongolia (MN)","Morocco (MA)", "Oman (OM-ar)", "Pakistan (PK)",
           "Panama (PA)","Peru (PE)", "Philippines (PH)","Puerto Rico (PR)", "Saudi Arabia (SA-ar)", "Singapore (SG)",
           "Sudan (SD-en)", "Thailand (TH)","Turkey (TR)", "Ukraine (UA)"]
as_countrys = ['aw','au','az','dm','bn','cl','co','cr','ec','fj','ge','hn','hk','id','kw','la','lb','my','mu',
               'mx','mn','ma','om/ar','pk','pa','pe','ph','pr','sa/ar','sg','sd/en','th','tr','ua']
gather_countrys = ['Aruba', 'Australia','Azerbaijan','British Dominica','Brunei','Chile',
                   'Colombia','Costa Rica','Ecuador','Fiji','Gerogia','Honduras','HongKong','Indonesia',
                   'Kuwait','Laos','Lebanon','Malaysia','Mauritius','Mexico','Mongolia','Morocco',
                   'Oman (ar)','Pakistan','Panama','Peru','Philippines','Puerto Rico','Saudi Arabia (SA-ar)',
                   'Singapore','Sudan (en)','Thailand','Turkey','Ukraine']

column_address = ['A','B','C','D','E','F','G','H','I','J','K','L','M',
                  'N','O','P','Q','R','S','T','U','V','W','X','Y','Z']

total_month_address = ["01. January", "02. February", "03. March", "04. April", "05. May", "06. June", "07. July",
                       "08. August", "09. September", "10. October", "11. November", "12. December"]
total_month_num = ["01월", "02월", "03월", "04월", "05월", "06월", "07월", "08월", "09월", "10월", "11월", "12월", ]

# 변경해야 하는 경로(최초 1회만 변경)
global_address = "C:/Dropbox (Ascent)/Ascent/2020/KIA/1. 모니터링/RAW/2020 Raw" 
organic_raw_address = "C:/Dropbox (Ascent)/Ascent/2020/KIA/1. 모니터링/RAW/Organic Traffic Raw"
gather_address = "C:/Dropbox (Ascent)/Ascent/2020/KIA/1. 모니터링/RAW/2020 Raw 정리"
final_address = "C:/Dropbox (Ascent)/Ascent/2020/KIA/1. 모니터링"

# 모니터링 월 설정 경로
month_address = total_month_address[monitering_month-1]
month_num = total_month_num[monitering_month-1]
gather_month = monitering_month # 10월달 관리대장 6.Device 수식 변경해 줘야 함

## ※ 파일 삭제 메소드(왠만하면 사용하지 말것)

In [4]:
# 삭제 메소드(파일 명만 교체해서 사용)
# for country in countrys:
#     os.remove(r"F:/OneDrive/2020 Raw"+"/"+country+"/"+month_address+"/"+
#               "1. On-page(Title, Description)1.xlsx")
# # 1번 파일명
# "1. On-page(Title, Description).xlsx"
# # 2번 파일명
# "2. Off-page(Total Backlinks).xlsx"
# # 2-1번 파일
# "2-1. Off-page(New Backlinks).xlsx"
# # 3번 파일명
# "3. Response Codes.xlsx"
# # 4번 파일명
# "4. Indexing HTTPS.xlsx"
# # 5번 파일명
# "5. Organic Traffic.xlsx"
# # 6번 파일명
# "6. Device Channel Traffic.xlsx"
# # 7번 파일명
# "7. Bounce Rate, Average Time Spent.xlsx"
# # 8번 파일명
# "8. Organic Keywords.xlsx"

## 1. On-page(Title, Description) - Title

In [5]:
count_num = 0

for as_country, country in zip(as_countrys, countrys):
    
    # 1차 불러오기
    title = pd.read_excel(global_address+"/"+country+"/"+month_address+"/"+"raw"+"/"+
                          "page_titles_all.xlsx", sheet_name = "1 - All")

    # 이상 URL 제거
    title_boolean_values = title["Address"].str.contains(pat="www.kia.com/"+as_country, regex=False)
    title_false_values = title["Address"][title_boolean_values == False]
    title.drop(title_false_values.index, inplace=True)
    
    # Total, Missing, Duplicate, Over 65, Below 30 변수 생성
    title_total = title['Address'].count()
    title_missing = title["Title 1"].isnull().sum()
    title_duplicate = title["Title 1"][title["Title 1"].duplicated(keep=False) == True].count()
    title_over = title["Title 1 Length"][title["Title 1 Length"]  > 65].count()
    title_below = title["Title 1 Length"][title["Title 1 Length"] < 30].count() - title_missing

    # 1차 저장
    with pd.ExcelWriter(global_address+"/"+country+"/"+month_address+"/"+
                        "1. On-page(Title, Description).xlsx",mode='a', engine='openpyxl') as writer:
        title.to_excel(writer, sheet_name = 'Title', index=False)

    # 2차 불러오기
    title_wb = openpyxl.load_workbook(global_address+"/"+country+"/"+month_address+"/"+
                                      "1. On-page(Title, Description).xlsx")
    title_ws = title_wb['Title']
    
    # 행 삽입
    title_ws.insert_rows(1)
    title_ws.insert_rows(1)
    title_ws.insert_rows(1)
    
    # total name, Missing name, Duplicate name, Over 156, Below 70 값 추가
    title_ws['A1'] = 'Total'
    title_ws['A1'].font = Font(bold=True)
    title_ws['A2'] = title_total
    
    title_ws['B1'] = 'Missing'
    title_ws['B1'].font = Font(bold=True)
    title_ws['B2'] = title_missing
    
    title_ws['C1'] = 'Duplicate'
    title_ws['C1'].font = Font(bold=True)
    title_ws['C2'] = title_duplicate
    
    title_ws['D1'] = 'Over 65'
    title_ws['D1'].font = Font(bold=True)
    title_ws['D2'] = title_over
    
    title_ws['E1'] = 'Below 30'
    title_ws['E1'].font = Font(bold=True)
    title_ws['E2'] = title_below
    
    # 2차 저장
    title_wb.save(global_address+"/"+country+"/"+month_address+"/"+"1. On-page(Title, Description).xlsx")
    
    # 결과
    count_num += 1
    print(count_num,". {} 나라의 {}, 1. On-page(Title, Description) Title 작업이 완료되었습니다.".format(country, month_address))

1 . Aruba (AW) 나라의 03. March, 1. On-page(Title, Description) Title 작업이 완료되었습니다.
2 . Australia (AU) 나라의 03. March, 1. On-page(Title, Description) Title 작업이 완료되었습니다.
3 . Azerbaijan (AZ) 나라의 03. March, 1. On-page(Title, Description) Title 작업이 완료되었습니다.
4 . British Dominica (DM) 나라의 03. March, 1. On-page(Title, Description) Title 작업이 완료되었습니다.
5 . Brunei (BN) 나라의 03. March, 1. On-page(Title, Description) Title 작업이 완료되었습니다.
6 . Chile (CL) 나라의 03. March, 1. On-page(Title, Description) Title 작업이 완료되었습니다.
7 . Colombia (CO) 나라의 03. March, 1. On-page(Title, Description) Title 작업이 완료되었습니다.
8 . Costa Rica (CR) 나라의 03. March, 1. On-page(Title, Description) Title 작업이 완료되었습니다.
9 . Ecuador (EC) 나라의 03. March, 1. On-page(Title, Description) Title 작업이 완료되었습니다.
10 . Fiji (FJ) 나라의 03. March, 1. On-page(Title, Description) Title 작업이 완료되었습니다.
11 . Georgia (GE) 나라의 03. March, 1. On-page(Title, Description) Title 작업이 완료되었습니다.
12 . Honduras (HN) 나라의 03. March, 1. On-page(Title, Description) Title 작업이 완료되었습니다.
13

## 1. On-page(Title, Description) - Meta Description

In [6]:
count_num = 0

for as_country, country in zip(as_countrys, countrys):
    
    # 1차 불러오기
    meta = pd.read_excel(global_address+"/"+country+"/"+month_address+"/"+"raw"+"/"+
                         "meta_description_all.xlsx",sheet_name = "1 - All")

    # 이상 URL 제거
    meta_boolean_values = meta["Address"].str.contains(pat="www.kia.com/"+as_country, regex=False)
    meta_false_values = meta["Address"][meta_boolean_values == False]
    meta.drop(index = meta_false_values.index, inplace=True)

    # Total, Missing, Duplicate, Over 156, Below70 변수 생성
    meta_total = meta['Address'].count()
    meta_missing = meta["Meta Description 1"].isnull().sum()
    meta_duplicate = meta["Meta Description 1"][meta["Meta Description 1"].duplicated(keep=False) == True].count()
    meta_over = meta["Meta Description 1 Length"][meta["Meta Description 1 Length"]  > 156].count()
    meta_below = meta["Meta Description 1 Length"][meta["Meta Description 1 Length"] < 70].count() - meta_missing

    # 1차 저장
    with pd.ExcelWriter(global_address+"/"+country+"/"+month_address+"/"+
                        "1. On-page(Title, Description).xlsx",mode='a', engine='openpyxl') as writer:
        meta.to_excel(writer, sheet_name = 'Meta', index=False)

    # 2차 불러오기
    meta_wb = openpyxl.load_workbook(global_address+"/"+country+"/"+month_address+"/"+
                                          "1. On-page(Title, Description).xlsx")
    meta_ws = meta_wb['Meta']
    
    # 행 삽입
    meta_ws.insert_rows(1)
    meta_ws.insert_rows(1)
    meta_ws.insert_rows(1)
    
    # total name, Missing name, Duplicate name, Over 156, Below 70 값 추가
    meta_ws['A1'] = 'Total'
    meta_ws['A1'].font = Font(bold=True)
    meta_ws['A2'] = meta_total
    
    meta_ws['B1'] = 'Missing'
    meta_ws['B1'].font = Font(bold=True)
    meta_ws['B2'] = meta_missing
    
    meta_ws['C1'] = 'Duplicate'
    meta_ws['C1'].font = Font(bold=True)
    meta_ws['C2'] = meta_duplicate
    
    meta_ws['D1'] = 'Over 156'
    meta_ws['D1'].font = Font(bold=True)
    meta_ws['D2'] = meta_over
    
    meta_ws['E1'] = 'Below 70'
    meta_ws['E1'].font = Font(bold=True)
    meta_ws['E2'] = meta_below
    
    # 2차 저장
    meta_wb.save(global_address+"/"+country+"/"+month_address+"/"+"1. On-page(Title, Description).xlsx")
    
    # 결과
    count_num += 1
    print(count_num,". {} 나라의 {}, 1. On-page(Title, Description) Description 작업이 완료되었습니다."
          .format(country, month_address))

1 . Aruba (AW) 나라의 03. March, 1. On-page(Title, Description) Description 작업이 완료되었습니다.
2 . Australia (AU) 나라의 03. March, 1. On-page(Title, Description) Description 작업이 완료되었습니다.
3 . Azerbaijan (AZ) 나라의 03. March, 1. On-page(Title, Description) Description 작업이 완료되었습니다.
4 . British Dominica (DM) 나라의 03. March, 1. On-page(Title, Description) Description 작업이 완료되었습니다.
5 . Brunei (BN) 나라의 03. March, 1. On-page(Title, Description) Description 작업이 완료되었습니다.
6 . Chile (CL) 나라의 03. March, 1. On-page(Title, Description) Description 작업이 완료되었습니다.
7 . Colombia (CO) 나라의 03. March, 1. On-page(Title, Description) Description 작업이 완료되었습니다.
8 . Costa Rica (CR) 나라의 03. March, 1. On-page(Title, Description) Description 작업이 완료되었습니다.
9 . Ecuador (EC) 나라의 03. March, 1. On-page(Title, Description) Description 작업이 완료되었습니다.
10 . Fiji (FJ) 나라의 03. March, 1. On-page(Title, Description) Description 작업이 완료되었습니다.
11 . Georgia (GE) 나라의 03. March, 1. On-page(Title, Description) Description 작업이 완료되었습니다.
12 . Honduras (HN) 나

## 1. On-page(Title, Description) - Summary

In [7]:
count_num = 0

for country in countrys:
    # 불러오기
    onpage_wb = openpyxl.load_workbook(global_address+"/"+country+"/"+month_address+"/"+
                                       "1. On-page(Title, Description).xlsx")
    onpage_ws = onpage_wb['On-Page SEO']
    title_ws = onpage_wb['Title']
    meta_ws = onpage_wb['Meta']
    
    # 도표 채우기
    onpage_ws['B2'] = country
    onpage_ws['B2'].font = Font(size=14, bold=True)
    
    onpage_ws['C6'] = title_ws.cell(row=2, column=1).value
    onpage_ws['C7'] = title_ws.cell(row=2, column=2).value
    onpage_ws['C8'] = title_ws.cell(row=2, column=3).value
    onpage_ws['C9'] = title_ws.cell(row=2, column=4).value
    onpage_ws['C10'] = title_ws.cell(row=2, column=5).value
    
    onpage_ws['C14'] = meta_ws.cell(row=2, column=1).value
    onpage_ws['C15'] = meta_ws.cell(row=2, column=2).value
    onpage_ws['C16'] = meta_ws.cell(row=2, column=3).value
    onpage_ws['C17'] = meta_ws.cell(row=2, column=4).value
    onpage_ws['C18'] = meta_ws.cell(row=2, column=5).value
    
    # 저장
    onpage_wb.save(global_address+"/"+country+"/"+month_address+"/"+"1. On-page(Title, Description).xlsx")
    
    # 결과
    count_num += 1
    print(count_num,". {} 나라의 {}, 1. On-page(Title, Description) On-Page SEO 작업이 완료되었습니다."
          .format(country, month_address))

1 . Aruba (AW) 나라의 03. March, 1. On-page(Title, Description) On-Page SEO 작업이 완료되었습니다.
2 . Australia (AU) 나라의 03. March, 1. On-page(Title, Description) On-Page SEO 작업이 완료되었습니다.
3 . Azerbaijan (AZ) 나라의 03. March, 1. On-page(Title, Description) On-Page SEO 작업이 완료되었습니다.
4 . British Dominica (DM) 나라의 03. March, 1. On-page(Title, Description) On-Page SEO 작업이 완료되었습니다.
5 . Brunei (BN) 나라의 03. March, 1. On-page(Title, Description) On-Page SEO 작업이 완료되었습니다.
6 . Chile (CL) 나라의 03. March, 1. On-page(Title, Description) On-Page SEO 작업이 완료되었습니다.
7 . Colombia (CO) 나라의 03. March, 1. On-page(Title, Description) On-Page SEO 작업이 완료되었습니다.
8 . Costa Rica (CR) 나라의 03. March, 1. On-page(Title, Description) On-Page SEO 작업이 완료되었습니다.
9 . Ecuador (EC) 나라의 03. March, 1. On-page(Title, Description) On-Page SEO 작업이 완료되었습니다.
10 . Fiji (FJ) 나라의 03. March, 1. On-page(Title, Description) On-Page SEO 작업이 완료되었습니다.
11 . Georgia (GE) 나라의 03. March, 1. On-page(Title, Description) On-Page SEO 작업이 완료되었습니다.
12 . Honduras (HN) 나

## 2. Off-page(Total Backlinks)

In [4]:
count_num = 0

for country in countrys:
    # 1차 불러오기(utf-16은 인코딩이 안됨)
#     total_backlinks = pd.read_csv(global_address+"/"+country+"/"+month_address+"/"+
#                                   "total_backlinks.csv" , encoding='utf-8')
    total_backlinks = pd.read_excel(global_address+"/"+country+"/"+month_address+"/"+"raw"+"/"+
                                  "2. total_backlinks_raw.xlsx")
    # total 합계 변수 생성
    total_backlinks_sum = total_backlinks['Total Backlinks'].sum()

    # 출력
    total_backlinks.to_excel(global_address+"/"+country+"/"+month_address+"/"+
                             "2. Off-page(Total Backlinks).xlsx", sheet_name = 'Off-page(Total Backlinks)', index=False)
    
    # 2차 불러오기
    total_backlinks_wb = openpyxl.load_workbook(global_address+"/"+country+"/"+month_address+"/"+
                                          "2. Off-page(Total Backlinks).xlsx")
    total_backlinks_ws = total_backlinks_wb['Off-page(Total Backlinks)']
    
    # 행 삽입
    total_backlinks_ws.insert_rows(1)
    total_backlinks_ws.insert_rows(1)
    total_backlinks_ws.insert_rows(1)
    
    # total name, 값 추가
    total_backlinks_ws['A1'] = 'Total'
    total_backlinks_ws['A1'].font = Font(bold=True)
    total_backlinks_ws['A2'] = total_backlinks_sum
    
    # 저장
    total_backlinks_wb.save(global_address+"/"+country+"/"+month_address+"/"+"2. Off-page(Total Backlinks).xlsx")
    
    # 결과
    count_num += 1
    print(count_num,". {} 나라의 {}, 2. Off-page(Total Backlinks) 작업이 완료되었습니다.".format(country, month_address))


1 . Aruba (AW) 나라의 03. March, 2. Off-page(Total Backlinks) 작업이 완료되었습니다.
2 . Australia (AU) 나라의 03. March, 2. Off-page(Total Backlinks) 작업이 완료되었습니다.
3 . Azerbaijan (AZ) 나라의 03. March, 2. Off-page(Total Backlinks) 작업이 완료되었습니다.
4 . British Dominica (DM) 나라의 03. March, 2. Off-page(Total Backlinks) 작업이 완료되었습니다.
5 . Brunei (BN) 나라의 03. March, 2. Off-page(Total Backlinks) 작업이 완료되었습니다.
6 . Chile (CL) 나라의 03. March, 2. Off-page(Total Backlinks) 작업이 완료되었습니다.
7 . Colombia (CO) 나라의 03. March, 2. Off-page(Total Backlinks) 작업이 완료되었습니다.
8 . Costa Rica (CR) 나라의 03. March, 2. Off-page(Total Backlinks) 작업이 완료되었습니다.
9 . Ecuador (EC) 나라의 03. March, 2. Off-page(Total Backlinks) 작업이 완료되었습니다.
10 . Fiji (FJ) 나라의 03. March, 2. Off-page(Total Backlinks) 작업이 완료되었습니다.
11 . Georgia (GE) 나라의 03. March, 2. Off-page(Total Backlinks) 작업이 완료되었습니다.
12 . Honduras (HN) 나라의 03. March, 2. Off-page(Total Backlinks) 작업이 완료되었습니다.
13 . Hong Kong (HK) 나라의 03. March, 2. Off-page(Total Backlinks) 작업이 완료되었습니다.
14 . Indonesia (ID) 나

## 2-1. Off-page(New Backlinks)
(csv 자체로 거르고 싶으면 utf-8로 다운 받아야 함)

In [9]:
count_num = 0

for country in countrys:
    # 1차 불러오기
    new_backlinks = pd.read_excel(global_address+"/"+country+"/"+month_address+"/"+"raw"+"/"+
                                  "2-1. new_backlinks_raw.xlsx")
    
    # total 합계 변수 생성
    new_backlinks_sum = new_backlinks['Total Backlinks'].sum()

    # 1차 저장
    new_backlinks.to_excel(global_address+"/"+country+"/"+month_address+"/"+"2-1. Off-page(New Backlinks).xlsx",
                           sheet_name = '2-1. Off-page(New Backlinks)', index=False)
    # 2차 불러오기
    new_backlinks_wb = openpyxl.load_workbook(global_address+"/"+country+"/"+month_address+"/"+
                                              "2-1. Off-page(New Backlinks).xlsx")
    new_backlinks_ws = new_backlinks_wb['2-1. Off-page(New Backlinks)']

    # 행 삽입
    new_backlinks_ws.insert_rows(1)
    new_backlinks_ws.insert_rows(1)
    new_backlinks_ws.insert_rows(1)

    # total name, 값 추가
    new_backlinks_ws['A1'] = 'Total'
    new_backlinks_ws['A1'].font = Font(bold=True)
    new_backlinks_ws['A2'] = new_backlinks_sum

    # 2차 저장
    new_backlinks_wb.save(global_address+"/"+country+"/"+month_address+"/"+"2-1. Off-page(New Backlinks).xlsx")

    # 결과
    count_num += 1
    print(count_num,". {} 나라의 {}, 2-1. Off-page(New Backlinks) 작업이 완료되었습니다."
          .format(country, month_address))

1 . Aruba (AW) 나라의 03. March, 2-1. Off-page(New Backlinks) 작업이 완료되었습니다.
2 . Australia (AU) 나라의 03. March, 2-1. Off-page(New Backlinks) 작업이 완료되었습니다.
3 . Azerbaijan (AZ) 나라의 03. March, 2-1. Off-page(New Backlinks) 작업이 완료되었습니다.
4 . British Dominica (DM) 나라의 03. March, 2-1. Off-page(New Backlinks) 작업이 완료되었습니다.
5 . Brunei (BN) 나라의 03. March, 2-1. Off-page(New Backlinks) 작업이 완료되었습니다.
6 . Chile (CL) 나라의 03. March, 2-1. Off-page(New Backlinks) 작업이 완료되었습니다.
7 . Colombia (CO) 나라의 03. March, 2-1. Off-page(New Backlinks) 작업이 완료되었습니다.
8 . Costa Rica (CR) 나라의 03. March, 2-1. Off-page(New Backlinks) 작업이 완료되었습니다.
9 . Ecuador (EC) 나라의 03. March, 2-1. Off-page(New Backlinks) 작업이 완료되었습니다.
10 . Fiji (FJ) 나라의 03. March, 2-1. Off-page(New Backlinks) 작업이 완료되었습니다.
11 . Georgia (GE) 나라의 03. March, 2-1. Off-page(New Backlinks) 작업이 완료되었습니다.
12 . Honduras (HN) 나라의 03. March, 2-1. Off-page(New Backlinks) 작업이 완료되었습니다.
13 . Hong Kong (HK) 나라의 03. March, 2-1. Off-page(New Backlinks) 작업이 완료되었습니다.
14 . Indonesia (ID) 나

## 3. Response Codes

In [10]:
count_num = 0

for country, as_country in zip(countrys, as_countrys):
    # Response Codes 불러오기
    response = pd.read_excel(global_address+"/"+country+"/"+month_address+"/"+"raw"+"/"+
                             "response_codes_all.xlsx", sheet_name = '1 - All')

    # 이상 URL 제거
    response_boolean_values = response['Address'].str.contains(pat="www.kia.com/"+as_country, regex=False)
    response_false_values = response['Address'][response_boolean_values == False]
    response.drop(index = response_false_values.index, inplace=True)

    # 피벗 변수 생성 및 저장
    response_pivot = response[['Status Code', 'Status', 'Address']].groupby(['Status Code', 'Status']).count()
    response_pivot.to_excel(global_address+"/"+country+"/"+month_address+"/"+
                            "3. Response Codes.xlsx", sheet_name = '피벗')

    # Response Codes 저장
    with pd.ExcelWriter(global_address+"/"+country+"/"+month_address+"/"+
                        "3. Response Codes.xlsx", mode='a', engine='openpyxl') as writer:
        response.to_excel(writer, sheet_name = 'Response Codes', index=False)

    # all_inlinks 불러오고 저장
    all_inlinks = pd.read_excel(global_address+"/"+country+"/"+month_address+"/"+
                                "raw"+"/"+"all_inlinks.xlsx", sheet_name = '1 - All Inlinks')

    with pd.ExcelWriter(global_address+"/"+country+"/"+month_address+"/"+
                        "3. Response Codes.xlsx", mode='a', engine='openpyxl') as writer:
        all_inlinks.to_excel(writer, sheet_name = '1 - All Inlinks', index=False)
        
    # 피벗 테이블 불러오기
    response_wb = openpyxl.load_workbook(global_address+"/"+country+"/"+month_address+"/"+
                                         "3. Response Codes.xlsx")
    response_ws = response_wb['피벗']
    
    # 피벗 셀 너비 설정
    response_ws.column_dimensions['A'].width = 12.00
    response_ws.column_dimensions['B'].width = 19.00
    response_ws.column_dimensions['C'].width = 8.00
    
    
    # 피벗 셀 테두리 설정
    border_box = Border(left = Side(border_style='thin', color='FF000000'), 
                        right = Side(border_style='thin', color='FF000000'), 
                        top = Side(border_style='thin', color='FF000000'), 
                        bottom = Side(border_style='thin', color='FF000000'),
                        diagonal = Side(border_style='thin', color='FF000000'),
                        diagonal_direction=0,
                        outline = Side(border_style='thin', color='FF000000'),
                        vertical = Side(border_style='thin', color='FF000000'),
                        horizontal = Side(border_style='thin', color='FF000000'))
    
    for sheet_column in range(1, response_pivot['Address'].count()+2):
        response_ws['C'+str(sheet_column)].border = border_box
    
    # 피벗 테이블 저장
    response_wb.save(global_address+"/"+country+"/"+month_address+"/"+"3. Response Codes.xlsx")
    
    # 결과
    count_num += 1
    print(count_num,". {} 나라의 {}, 3. Response Codes 작업이 완료되었습니다.".format(country, month_address))

1 . Aruba (AW) 나라의 03. March, 3. Response Codes 작업이 완료되었습니다.
2 . Australia (AU) 나라의 03. March, 3. Response Codes 작업이 완료되었습니다.
3 . Azerbaijan (AZ) 나라의 03. March, 3. Response Codes 작업이 완료되었습니다.
4 . British Dominica (DM) 나라의 03. March, 3. Response Codes 작업이 완료되었습니다.
5 . Brunei (BN) 나라의 03. March, 3. Response Codes 작업이 완료되었습니다.
6 . Chile (CL) 나라의 03. March, 3. Response Codes 작업이 완료되었습니다.
7 . Colombia (CO) 나라의 03. March, 3. Response Codes 작업이 완료되었습니다.
8 . Costa Rica (CR) 나라의 03. March, 3. Response Codes 작업이 완료되었습니다.
9 . Ecuador (EC) 나라의 03. March, 3. Response Codes 작업이 완료되었습니다.
10 . Fiji (FJ) 나라의 03. March, 3. Response Codes 작업이 완료되었습니다.
11 . Georgia (GE) 나라의 03. March, 3. Response Codes 작업이 완료되었습니다.
12 . Honduras (HN) 나라의 03. March, 3. Response Codes 작업이 완료되었습니다.
13 . Hong Kong (HK) 나라의 03. March, 3. Response Codes 작업이 완료되었습니다.
14 . Indonesia (ID) 나라의 03. March, 3. Response Codes 작업이 완료되었습니다.
15 . Kuwait (KW) 나라의 03. March, 3. Response Codes 작업이 완료되었습니다.
16 . Laos (LA) 나라의 03. March, 3. Re

## 4. Indexing HTTPS
(csv확장자에서 xlsx로 바뀜)

In [11]:
count_num = 0

for country in countrys:
    
    # 1차 불러오기
    indexing_https = pd.read_excel(global_address+"/"+country+"/"+month_address+"/"+"raw"+"/"+
                                   '4. indexing_https_raw.xlsx', sheet_name='Table')
    
    # 해당 Type별 숫자 계산
    string_sort_1 = indexing_https['Pages'][indexing_https['Type'] =='Submitted and indexed'].values[0]
    string_sort_2 = indexing_https['Pages'][indexing_https['Type'] =='Indexed, not submitted in sitemap'].values[0]
    
    # 1차 저장
    indexing_https.to_excel(global_address+"/"+country+"/"+month_address+"/"+'4. Indexing HTTPS.xlsx',
                            sheet_name='Table')
    
    # 2차 불러오기
    indexing_wb = openpyxl.load_workbook(global_address+"/"+country+"/"+month_address+"/"+
                                         '4. Indexing HTTPS.xlsx')
    indexing_ws = indexing_wb['Table']
    
    # 가공
    indexing_ws['G1'] = 'Total Indexed'
    indexing_ws['G2'] = 'Submitted Web Indexed'
    
    indexing_ws['H1'] = string_sort_1 + string_sort_2
    indexing_ws['H2'] = string_sort_1
    
    # 2차 저장
    indexing_wb.save(global_address+"/"+country+"/"+month_address+"/"+'4. Indexing HTTPS.xlsx')
    
    # 결과
    count_num += 1
    print(count_num,". {} 나라의 {}, 4. Indexing HTTPS 작업이 완료되었습니다.".format(country, month_address))

1 . Aruba (AW) 나라의 03. March, 4. Indexing HTTPS 작업이 완료되었습니다.
2 . Australia (AU) 나라의 03. March, 4. Indexing HTTPS 작업이 완료되었습니다.
3 . Azerbaijan (AZ) 나라의 03. March, 4. Indexing HTTPS 작업이 완료되었습니다.
4 . British Dominica (DM) 나라의 03. March, 4. Indexing HTTPS 작업이 완료되었습니다.
5 . Brunei (BN) 나라의 03. March, 4. Indexing HTTPS 작업이 완료되었습니다.
6 . Chile (CL) 나라의 03. March, 4. Indexing HTTPS 작업이 완료되었습니다.
7 . Colombia (CO) 나라의 03. March, 4. Indexing HTTPS 작업이 완료되었습니다.
8 . Costa Rica (CR) 나라의 03. March, 4. Indexing HTTPS 작업이 완료되었습니다.
9 . Ecuador (EC) 나라의 03. March, 4. Indexing HTTPS 작업이 완료되었습니다.
10 . Fiji (FJ) 나라의 03. March, 4. Indexing HTTPS 작업이 완료되었습니다.
11 . Georgia (GE) 나라의 03. March, 4. Indexing HTTPS 작업이 완료되었습니다.
12 . Honduras (HN) 나라의 03. March, 4. Indexing HTTPS 작업이 완료되었습니다.
13 . Hong Kong (HK) 나라의 03. March, 4. Indexing HTTPS 작업이 완료되었습니다.
14 . Indonesia (ID) 나라의 03. March, 4. Indexing HTTPS 작업이 완료되었습니다.
15 . Kuwait (KW) 나라의 03. March, 4. Indexing HTTPS 작업이 완료되었습니다.
16 . Laos (LA) 나라의 03. March, 4. In

## 5. Organic traffic(preprocessing 1)

In [12]:
# 불러오기
organic_traffic_wb = openpyxl.load_workbook(organic_raw_address+"/"+"5. organic_traffic_raw.xlsx")
organic_traffic_ws = organic_traffic_wb['Organic Traffic Raw - Web - Glo']

# 행 삭제, 행 추가
for i in range(9):
    organic_traffic_ws.delete_rows(115)
for i in range(14):
    organic_traffic_ws.delete_rows(1)
    
# 컬럼명 만들기
organic_traffic_ws['C1'] = 'Visits'
organic_traffic_ws['D1'] = 'Search Engines'
organic_traffic_ws['E1'] = '[ibank]Paid Marketing_New'
organic_traffic_ws['F1'] = '[ibank]Non-Paid Marketing_New'

# 저장
organic_traffic_wb.save(organic_raw_address+"/"+"Organic Traffic Raw_"+month_num+".xlsx")

# 결과
print("{}, Organic Traffic Raw 1차 전처리 작업이 완료되었습니다.".format(month_address))

03. March, Organic Traffic Raw 1차 전처리 작업이 완료되었습니다.


## 5. Organic traffic(preprocessing 2)

In [13]:
# 불러오기 1차
organic_traffic_pre = pd.read_excel(organic_raw_address+"/"+"Organic Traffic Raw_"+month_num+".xlsx",
                                    sheet_name = 'Organic Traffic Raw - Web - Glo')

# Country code 대문자 변환
organic_traffic_pre['Country code'] = organic_traffic_pre['Country code'].str.upper()

# B2 열 공백 행 삭제
duplicate_false_index= organic_traffic_pre['Country code'][organic_traffic_pre['Country code'].
                                                           duplicated(keep='last') == True].index
organic_traffic_pre.drop(index=duplicate_false_index, inplace=True)

# 저장 1차
organic_traffic_pre.to_excel(organic_raw_address+"/"+"Organic Traffic Raw_"+month_num+".xlsx",
                             sheet_name = 'Organic Traffic Raw - Web - Glo',index=False)

# 결과
print("{}, Organic Traffic Raw 2차 전처리 작업이 완료되었습니다.".format(month_address))

03. March, Organic Traffic Raw 2차 전처리 작업이 완료되었습니다.


## 5. Organic traffic(preprocessing 3)

In [14]:
# 불러오기 1차
organic_traffic_pre_wb = openpyxl.load_workbook(organic_raw_address+"/"+"Organic Traffic Raw_"+
                                                month_num+".xlsx")
organic_traffic_pre_ws = organic_traffic_pre_wb['Organic Traffic Raw - Web - Glo']


# 가공
organic_traffic_pre_ws.delete_rows(11)
organic_traffic_pre_ws.insert_rows(29)
organic_traffic_pre_ws.insert_rows(24)

organic_traffic_pre_ws['A24'] = organic_traffic_pre_ws.cell(row=36, column=1).value
organic_traffic_pre_ws['B24'] = organic_traffic_pre_ws.cell(row=36, column=2).value
organic_traffic_pre_ws['C24'] = organic_traffic_pre_ws.cell(row=36, column=3).value
organic_traffic_pre_ws['D24'] = organic_traffic_pre_ws.cell(row=36, column=4).value
organic_traffic_pre_ws['E24'] = organic_traffic_pre_ws.cell(row=36, column=5).value
organic_traffic_pre_ws['F24'] = organic_traffic_pre_ws.cell(row=36, column=6).value

organic_traffic_pre_ws['A30'] = organic_traffic_pre_ws.cell(row=37, column=1).value
organic_traffic_pre_ws['B30'] = organic_traffic_pre_ws.cell(row=37, column=2).value
organic_traffic_pre_ws['C30'] = organic_traffic_pre_ws.cell(row=37, column=3).value
organic_traffic_pre_ws['D30'] = organic_traffic_pre_ws.cell(row=37, column=4).value
organic_traffic_pre_ws['E30'] = organic_traffic_pre_ws.cell(row=37, column=5).value
organic_traffic_pre_ws['F30'] = organic_traffic_pre_ws.cell(row=37, column=6).value

organic_traffic_pre_ws.delete_rows(37)
organic_traffic_pre_ws.delete_rows(36)

# 저장 1차
organic_traffic_pre_wb.save(organic_raw_address+"/"+"Organic Traffic Raw_"+month_num+".xlsx")

# Raw 불러오기 2차
organic_traffic_pre_raw = pd.read_excel(organic_raw_address+"/"+"5. organic_traffic_raw.xlsx")

# Raw 저장하기 2차
with pd.ExcelWriter(organic_raw_address+"/"+"Organic Traffic Raw_"+month_num+".xlsx", 
                    mode='a', engine='openpyxl') as writer:
        organic_traffic_pre_raw.to_excel(writer, sheet_name = 'Raw', index=False)

# 결과
print("{}, Organic Traffic Raw 3차 전처리 작업이 완료되었습니다.".format(month_address))

03. March, Organic Traffic Raw 3차 전처리 작업이 완료되었습니다.


## 5. Organic traffic(separate)

In [15]:
count_num = 0
# ws_row = 1 (AW 부터 시작시)
ws_row = 1


for country in countrys:
    # 불러오기 1차
    organic_sep_wb = openpyxl.load_workbook(organic_raw_address+"/"+"Organic Traffic Raw_"+month_num+".xlsx")
    organic_sep_ws = organic_sep_wb['Organic Traffic Raw - Web - Glo']
    
        
    # sheet 이름 변경
    organic_sep_ws.title = 'Organic Traffic Raw_'+month_num
    
    # sheet 생성
    organic_sep_wb.remove(organic_sep_wb['Raw'])
    organic_sheet = organic_sep_wb.create_sheet('Organic Traffic',0)
    
    # sheet 내용 작성
    organic_sheet['B2'] = 'Country code'
    organic_sheet['C2'] = 'Month'
    organic_sheet['D2'] = 'Visits'
    organic_sheet['E2'] = 'Search Engines Visits Total'
    organic_sheet['F2'] = 'Paid Search Engines'
    organic_sheet['G2'] = 'Organic Search Engines'

    ws_row = ws_row + 1

    organic_sheet['B3'] = organic_sep_ws.cell(row=ws_row, column=1).value
    organic_sheet['C3'] = str(organic_sep_ws.cell(row=ws_row, column=2).value)[0:7]
    organic_sheet['D3'] = organic_sep_ws.cell(row=ws_row, column=3).value
    organic_sheet['E3'] = organic_sep_ws.cell(row=ws_row, column=4).value
    organic_sheet['F3'] = organic_sep_ws.cell(row=ws_row, column=5).value
    organic_sheet['G3'] = organic_sep_ws.cell(row=ws_row, column=6).value

    # Organic Traffic Raw_00월 sheet 삭제
    organic_sep_wb.remove(organic_sep_wb['Organic Traffic Raw_'+month_num])
    
    # 셀 색 채우기 
    organic_sheet['B2'].fill = PatternFill(patternType='solid', fgColor=Color('FFC000'))
    organic_sheet['C2'].fill = PatternFill(patternType='solid', fgColor=Color('FFC000'))
    organic_sheet['D2'].fill = PatternFill(patternType='solid', fgColor=Color('FFC000'))
    organic_sheet['E2'].fill = PatternFill(patternType='solid', fgColor=Color('FFC000'))
    organic_sheet['F2'].fill = PatternFill(patternType='solid', fgColor=Color('FFC000'))
    organic_sheet['G2'].fill = PatternFill(patternType='solid', fgColor=Color('FFC000'))
    organic_sheet['G3'].fill = PatternFill(patternType='solid', fgColor=Color('D8E4BC'))
    
    # 글자 진하게
    organic_sheet['B2'].font = Font(bold=True)
    organic_sheet['C2'].font = Font(bold=True)
    organic_sheet['D2'].font = Font(bold=True)
    organic_sheet['E2'].font = Font(bold=True)
    organic_sheet['F2'].font = Font(bold=True)
    organic_sheet['G2'].font = Font(bold=True)
    
    # 셀 너비 조정
    organic_sheet.column_dimensions['B'].width = 13.00
    organic_sheet.column_dimensions['C'].width = 17.20
    organic_sheet.column_dimensions['D'].width = 15.00
    organic_sheet.column_dimensions['E'].width = 25.30
    organic_sheet.column_dimensions['F'].width = 19.00
    organic_sheet.column_dimensions['G'].width = 22.30
    
    # 가운데 정렬
    organic_sheet['B2'].alignment = Alignment(horizontal='center', vertical='center')
    organic_sheet['C2'].alignment = Alignment(horizontal='center', vertical='center')
    organic_sheet['D2'].alignment = Alignment(horizontal='center', vertical='center')
    organic_sheet['E2'].alignment = Alignment(horizontal='center', vertical='center')
    organic_sheet['F2'].alignment = Alignment(horizontal='center', vertical='center')
    organic_sheet['G2'].alignment = Alignment(horizontal='center', vertical='center')
    
    organic_sheet['B3'].alignment = Alignment(horizontal='center', vertical='center')
    organic_sheet['C3'].alignment = Alignment(horizontal='center', vertical='center')
    organic_sheet['D3'].alignment = Alignment(horizontal='center', vertical='center')
    organic_sheet['E3'].alignment = Alignment(horizontal='center', vertical='center')
    organic_sheet['F3'].alignment = Alignment(horizontal='center', vertical='center')
    organic_sheet['G3'].alignment = Alignment(horizontal='center', vertical='center')
    
    # 셀 테두리 설정
    border_box = Border(left = Side(border_style='thin', color='FF000000'), 
                        right = Side(border_style='thin', color='FF000000'), 
                        top = Side(border_style='thin', color='FF000000'), 
                        bottom = Side(border_style='thin', color='FF000000'),
                        diagonal = Side(border_style='thin', color='FF000000'),
                        diagonal_direction=0,
                        outline = Side(border_style='thin', color='FF000000'),
                        vertical = Side(border_style='thin', color='FF000000'),
                        horizontal = Side(border_style='thin', color='FF000000'))
    
    organic_sheet['B2'].border = border_box
    organic_sheet['C2'].border = border_box
    organic_sheet['D2'].border = border_box
    organic_sheet['E2'].border = border_box
    organic_sheet['F2'].border = border_box
    organic_sheet['G2'].border = border_box
    
    organic_sheet['B3'].border = border_box
    organic_sheet['C3'].border = border_box
    organic_sheet['D3'].border = border_box
    organic_sheet['E3'].border = border_box
    organic_sheet['F3'].border = border_box
    organic_sheet['G3'].border = border_box
    
    # 저장 1차
    organic_sep_wb.save(global_address+"/"+country+"/"+month_address+"/"+"5. Organic Traffic.xlsx")
    
    # 결과
    count_num += 1
    print(count_num,".{}, {} 빈 값 excel 파일 뿌리기 작업이 완료되었습니다.".format(month_address, country))

1 .03. March, Aruba (AW) 빈 값 excel 파일 뿌리기 작업이 완료되었습니다.
2 .03. March, Australia (AU) 빈 값 excel 파일 뿌리기 작업이 완료되었습니다.
3 .03. March, Azerbaijan (AZ) 빈 값 excel 파일 뿌리기 작업이 완료되었습니다.
4 .03. March, British Dominica (DM) 빈 값 excel 파일 뿌리기 작업이 완료되었습니다.
5 .03. March, Brunei (BN) 빈 값 excel 파일 뿌리기 작업이 완료되었습니다.
6 .03. March, Chile (CL) 빈 값 excel 파일 뿌리기 작업이 완료되었습니다.
7 .03. March, Colombia (CO) 빈 값 excel 파일 뿌리기 작업이 완료되었습니다.
8 .03. March, Costa Rica (CR) 빈 값 excel 파일 뿌리기 작업이 완료되었습니다.
9 .03. March, Ecuador (EC) 빈 값 excel 파일 뿌리기 작업이 완료되었습니다.
10 .03. March, Fiji (FJ) 빈 값 excel 파일 뿌리기 작업이 완료되었습니다.
11 .03. March, Georgia (GE) 빈 값 excel 파일 뿌리기 작업이 완료되었습니다.
12 .03. March, Honduras (HN) 빈 값 excel 파일 뿌리기 작업이 완료되었습니다.
13 .03. March, Hong Kong (HK) 빈 값 excel 파일 뿌리기 작업이 완료되었습니다.
14 .03. March, Indonesia (ID) 빈 값 excel 파일 뿌리기 작업이 완료되었습니다.
15 .03. March, Kuwait (KW) 빈 값 excel 파일 뿌리기 작업이 완료되었습니다.
16 .03. March, Laos (LA) 빈 값 excel 파일 뿌리기 작업이 완료되었습니다.
17 .03. March, Lebanon (LB) 빈 값 excel 파일 뿌리기 작업이 완료되었습니다.
18 .03. Marc

## 6. Device Channel Traffic
(% 뒤의 숫자를 자르지 말아야 함)

In [79]:
count_num = 0

for country in countrys:
    
    # 불러오기
    device_wb = openpyxl.load_workbook(global_address+"/"+country+"/"+month_address+"/"+"raw"+"/"+
                                       "6. device_raw.xlsx")
    device_ws = device_wb['Worksheet']

    # sheet 내용 작성
    device_ws['F18'] = 'mobile'
    device_ws['G18'] = 'desktop'
    
    mobile_sum = []
    desktop_sum = []
    for row_idx in range(1,47):
        if device_ws.cell(row=row_idx, column=2).value == 'Mobile Phone':
            mobile_sum.append(device_ws.cell(row=row_idx, column=4).value)
        elif device_ws.cell(row=row_idx, column=2).value == 'Tablet':
            mobile_sum.append(device_ws.cell(row=row_idx, column=4).value)
        elif device_ws.cell(row=row_idx, column=2).value == 'Television':
            desktop_sum.append(device_ws.cell(row=row_idx, column=4).value)
        elif device_ws.cell(row=row_idx, column=2).value == 'Media Player':
            desktop_sum.append(device_ws.cell(row=row_idx, column=4).value)
        elif device_ws.cell(row=row_idx, column=2).value == 'Set-top Box':
            desktop_sum.append(device_ws.cell(row=row_idx, column=4).value)
        elif device_ws.cell(row=row_idx, column=2).value == 'Gaming Console':
            desktop_sum.append(device_ws.cell(row=row_idx, column=4).value)
        elif device_ws.cell(row=row_idx, column=2).value == 'Other':
            desktop_sum.append(device_ws.cell(row=row_idx, column=4).value)
        elif device_ws.cell(row=row_idx, column=2).value == 'EReader':
            desktop_sum.append(device_ws.cell(row=row_idx, column=4).value)
        
    device_ws['F19'] = "{}%".format((sum(mobile_sum))*100)
    device_ws['G19'] = "{}%".format((sum(desktop_sum))*100)

    # sheet 색 변경
    for i in range(7,15):
        device_ws['A'+str(i)].fill = PatternFill(patternType='solid', fgColor=Color('F2F2F2'))
        device_ws['B'+str(i)].fill = PatternFill(patternType='solid', fgColor=Color('F2F2F2'))
        device_ws['C'+str(i)].fill = PatternFill(patternType='solid', fgColor=Color('F2F2F2'))
        device_ws['D'+str(i)].fill = PatternFill(patternType='solid', fgColor=Color('F2F2F2'))
    
    # 저장
    device_wb.save(global_address+"/"+country+"/"+month_address+"/"+"6. Device Channel Traffic.xlsx")
    
    # 결과
    count_num += 1
    print(count_num, ". {} 나라의 {}, 6. Device Channel Traffic 작업이 완료되었습니다.".format(country, month_address))

1 Aruba (AW) 나라의 03. March, 6. Device Channel Traffic 작업이 완료되었습니다.
2 Australia (AU) 나라의 03. March, 6. Device Channel Traffic 작업이 완료되었습니다.
3 Azerbaijan (AZ) 나라의 03. March, 6. Device Channel Traffic 작업이 완료되었습니다.
4 British Dominica (DM) 나라의 03. March, 6. Device Channel Traffic 작업이 완료되었습니다.
5 Brunei (BN) 나라의 03. March, 6. Device Channel Traffic 작업이 완료되었습니다.
6 Chile (CL) 나라의 03. March, 6. Device Channel Traffic 작업이 완료되었습니다.
7 Colombia (CO) 나라의 03. March, 6. Device Channel Traffic 작업이 완료되었습니다.
8 Costa Rica (CR) 나라의 03. March, 6. Device Channel Traffic 작업이 완료되었습니다.
9 Ecuador (EC) 나라의 03. March, 6. Device Channel Traffic 작업이 완료되었습니다.
10 Fiji (FJ) 나라의 03. March, 6. Device Channel Traffic 작업이 완료되었습니다.
11 Georgia (GE) 나라의 03. March, 6. Device Channel Traffic 작업이 완료되었습니다.
12 Honduras (HN) 나라의 03. March, 6. Device Channel Traffic 작업이 완료되었습니다.
13 Hong Kong (HK) 나라의 03. March, 6. Device Channel Traffic 작업이 완료되었습니다.
14 Indonesia (ID) 나라의 03. March, 6. Device Channel Traffic 작업이 완료되었습니다.
15 Kuwait (KW)

## 7. Bounce Rate, Average Time Spent
(굳이 실행시킬 필요 없을 것 같음)

In [17]:
count_num = 0

for country in countrys:
        
    # 불러오기
    bounce_wb = openpyxl.load_workbook(global_address+"/"+country+"/"+month_address+"/"+"raw"+"/"+
                                       "7. bounce_average_raw.xlsx")
    bounce_ws = bounce_wb['Worksheet']
    
    # sheet 색 변경
    for i in range(7,12):
        bounce_ws['A'+str(i)].fill = PatternFill(patternType='solid', fgColor=Color('F2F2F2'))
        bounce_ws['B'+str(i)].fill = PatternFill(patternType='solid', fgColor=Color('F2F2F2'))
        bounce_ws['C'+str(i)].fill = PatternFill(patternType='solid', fgColor=Color('F2F2F2'))
        bounce_ws['D'+str(i)].fill = PatternFill(patternType='solid', fgColor=Color('F2F2F2'))
        bounce_ws['E'+str(i)].fill = PatternFill(patternType='solid', fgColor=Color('F2F2F2'))
        
    # 저장하기
    bounce_wb.save(global_address+"/"+country+"/"+month_address+"/"+"7. Bounce Rate, Average Time Spent.xlsx")
    
    # 결과
    count_num += 1
    print(count_num,". {} 나라의 {}, 7. Bounce Rate, Average Time Spent 작업이 완료되었습니다.".format(country, month_address))

1 . Aruba (AW) 나라의 03. March, 7. Bounce Rate, Average Time Spent 작업이 완료되었습니다.
2 . Australia (AU) 나라의 03. March, 7. Bounce Rate, Average Time Spent 작업이 완료되었습니다.
3 . Azerbaijan (AZ) 나라의 03. March, 7. Bounce Rate, Average Time Spent 작업이 완료되었습니다.
4 . British Dominica (DM) 나라의 03. March, 7. Bounce Rate, Average Time Spent 작업이 완료되었습니다.
5 . Brunei (BN) 나라의 03. March, 7. Bounce Rate, Average Time Spent 작업이 완료되었습니다.
6 . Chile (CL) 나라의 03. March, 7. Bounce Rate, Average Time Spent 작업이 완료되었습니다.
7 . Colombia (CO) 나라의 03. March, 7. Bounce Rate, Average Time Spent 작업이 완료되었습니다.
8 . Costa Rica (CR) 나라의 03. March, 7. Bounce Rate, Average Time Spent 작업이 완료되었습니다.
9 . Ecuador (EC) 나라의 03. March, 7. Bounce Rate, Average Time Spent 작업이 완료되었습니다.
10 . Fiji (FJ) 나라의 03. March, 7. Bounce Rate, Average Time Spent 작업이 완료되었습니다.
11 . Georgia (GE) 나라의 03. March, 7. Bounce Rate, Average Time Spent 작업이 완료되었습니다.
12 . Honduras (HN) 나라의 03. March, 7. Bounce Rate, Average Time Spent 작업이 완료되었습니다.
13 . Hong Kong (HK) 나라의 03

## 8. Organic Keywords

In [80]:
count_num = 0

for country in countrys:
    # 불러오기
    organic_keywords = pd.read_excel(global_address+"/"+country+"/"+month_address+"/"+"raw"+"/"+
                                     "8. organic_keywords_raw.xlsx")
    
    # Total 개수, Click 0 초과, Click 0 초과 포지션 평균 변수 생성
    organic_keywords_total = organic_keywords['Query'].count()
    organic_keywords_over = organic_keywords['Clicks'][organic_keywords['Clicks'] > 0].count()
    organic_keywords_average = organic_keywords['Position'][organic_keywords['Clicks'] > 0].mean()
    
    # 2차 불러오기
    organic_keywords_wb = openpyxl.load_workbook(global_address+"/"+country+"/"+month_address+"/"+
                                                 "raw"+"/"+"8. organic_keywords_raw.xlsx")
    organic_keywords_ws = organic_keywords_wb['Sheet1']
    
    # sheet 이름 변경
    organic_keywords_ws.title = 'HTTPS'
    
    # 행 삽입
    organic_keywords_ws.insert_rows(1)
    organic_keywords_ws.insert_rows(1)
    organic_keywords_ws.insert_rows(1)
    
    # Total , Click 0 over, Click 0 over Average / name, 값 추가
    organic_keywords_ws['A1'] = 'Total'
    organic_keywords_ws['A1'].font = Font(bold=True)
    organic_keywords_ws['A2'] = organic_keywords_total
    
    organic_keywords_ws['B1'] = 'Click 0 over'
    organic_keywords_ws['B1'].font = Font(bold=True)
    organic_keywords_ws['B2'] = organic_keywords_over
    
    organic_keywords_ws['E1'] = 'Average'
    organic_keywords_ws['E1'].font = Font(bold=True)
    organic_keywords_ws['E2'] = '{0:.1f}'.format(organic_keywords_average)
    organic_keywords_ws['E2'].fill = PatternFill(patternType='solid', fgColor=Color('FFFF00')) 
    
    # 2차 저장
    organic_keywords_wb.save(global_address+"/"+country+"/"+month_address+"/"+"8. Organic Keywords.xlsx")
    
    # 결과
    count_num += 1
    print(count_num,". {} 나라의 {}, 8. Organic Keywords 작업이 완료되었습니다.".format(country, month_address))

1 . Aruba (AW) 나라의 03. March, 8. Organic Keywords 작업이 완료되었습니다.
2 . Australia (AU) 나라의 03. March, 8. Organic Keywords 작업이 완료되었습니다.
3 . Azerbaijan (AZ) 나라의 03. March, 8. Organic Keywords 작업이 완료되었습니다.
4 . British Dominica (DM) 나라의 03. March, 8. Organic Keywords 작업이 완료되었습니다.
5 . Brunei (BN) 나라의 03. March, 8. Organic Keywords 작업이 완료되었습니다.
6 . Chile (CL) 나라의 03. March, 8. Organic Keywords 작업이 완료되었습니다.
7 . Colombia (CO) 나라의 03. March, 8. Organic Keywords 작업이 완료되었습니다.
8 . Costa Rica (CR) 나라의 03. March, 8. Organic Keywords 작업이 완료되었습니다.
9 . Ecuador (EC) 나라의 03. March, 8. Organic Keywords 작업이 완료되었습니다.
10 . Fiji (FJ) 나라의 03. March, 8. Organic Keywords 작업이 완료되었습니다.
11 . Georgia (GE) 나라의 03. March, 8. Organic Keywords 작업이 완료되었습니다.
12 . Honduras (HN) 나라의 03. March, 8. Organic Keywords 작업이 완료되었습니다.
13 . Hong Kong (HK) 나라의 03. March, 8. Organic Keywords 작업이 완료되었습니다.
14 . Indonesia (ID) 나라의 03. March, 8. Organic Keywords 작업이 완료되었습니다.
15 . Kuwait (KW) 나라의 03. March, 8. Organic Keywords 작업이 완료되었습니다.
16 . 

## 나라별 Raw 데이터 정리(월별)

In [5]:
count_num = 0

for country, gather_country in zip(countrys, gather_countrys):
    
    # 가공 파일 불러오기
    gather_2_wb = openpyxl.load_workbook(global_address+"/"+country+"/"+month_address+"/"+
                                         "2. Off-page(Total Backlinks).xlsx")
    gather_2_ws = gather_2_wb['Off-page(Total Backlinks)']

    gather_2_1_wb = openpyxl.load_workbook(global_address+"/"+country+"/"+month_address+"/"+
                                           "2-1. Off-page(New Backlinks).xlsx")
    gather_2_1_ws = gather_2_1_wb['2-1. Off-page(New Backlinks)']

    gather_4_wb = openpyxl.load_workbook(global_address+"/"+country+"/"+month_address+"/"+
                                         "4. Indexing HTTPS.xlsx")
    gather_4_ws = gather_4_wb['Table']

    gather_5_wb = openpyxl.load_workbook(global_address+"/"+country+"/"+month_address+"/"+
                                         "5. Organic Traffic.xlsx")
    gather_5_ws = gather_5_wb['Organic Traffic']

    gather_6_wb = openpyxl.load_workbook(global_address+"/"+country+"/"+month_address+"/"+
                                         "6. Device Channel Traffic.xlsx")
    gather_6_ws = gather_6_wb['Worksheet']

    gather_7_wb = openpyxl.load_workbook(global_address+"/"+country+"/"+month_address+"/"+
                                         "7. Bounce Rate, Average Time Spent.xlsx")
    gather_7_ws = gather_7_wb['Worksheet']

    gather_8_wb = openpyxl.load_workbook(global_address+"/"+country+"/"+month_address+"/"+
                                         "8. Organic Keywords.xlsx")
    gather_8_ws = gather_8_wb['HTTPS']
    
    # Raw 정리 파일 불러오기
    gather_workbook = openpyxl.load_workbook(gather_address+"/"+gather_country+".xlsx")
    
    gather_2_offpage_sheet = gather_workbook['Off-Page']
    gather_4_index_sheet = gather_workbook['Index']
    gather_5_organic_sheet = gather_workbook['Organic']
    gather_6_device_sheet = gather_workbook['Device']
    gather_7_Bounce_sheet = gather_workbook['Bounce&Time']
    gather_8_kw_sheet = gather_workbook['KW']
    
    # 데이터 삽입

    gather_2_offpage_sheet['E'+str(gather_month+41)] = gather_2_ws.cell(row=2, column=1).value
    gather_2_offpage_sheet['D'+str(gather_month+41)] = gather_2_1_ws.cell(row=2, column=1).value
    
    gather_4_index_sheet['O'+str(gather_month+41)] =gather_4_ws.cell(row=1, column=8).value
    gather_4_index_sheet['T'+str(gather_month+41)] = gather_4_ws.cell(row=2, column=8).value
    
    gather_5_organic_sheet['D'+str(gather_month+41)] = gather_5_ws.cell(row=3, column=7).value
    
    gather_6_device_sheet['D'+str(gather_month+41)] = gather_6_ws.cell(row=19, column=6).value
    gather_6_device_sheet['E'+str(gather_month+41)] = gather_6_ws.cell(row=19, column=7).value
    
    gather_7_Bounce_sheet['D'+str(gather_month+41)] = "{} AM".format(gather_7_ws.cell(row=17, column=3).value)
    gather_7_Bounce_sheet['E'+str(gather_month+41)] = gather_7_ws.cell(row=17, column=4).value
    
    gather_8_kw_sheet['D'+str(gather_month+41)] = gather_8_ws.cell(row=2, column=1).value
    gather_8_kw_sheet['E'+str(gather_month+41)] = gather_8_ws.cell(row=2, column=2).value
    gather_8_kw_sheet['I'+str(gather_month+41)] = gather_8_ws.cell(row=2, column=5).value
    
    # Raw 정리 파일 저장
    gather_workbook.save(gather_address+"/"+gather_country+"_"+month_num+".xlsx")
    
    # 결과
    count_num += 1
    print(count_num,". {} 나라의 {}, 나라별 Raw 데이터 정리 작업이 완료되었습니다."
          .format(country, month_address))


1 . Aruba (AW) 나라의 03. March, 나라별 Raw 데이터 정리 작업이 완료되었습니다.
2 . Australia (AU) 나라의 03. March, 나라별 Raw 데이터 정리 작업이 완료되었습니다.
3 . Azerbaijan (AZ) 나라의 03. March, 나라별 Raw 데이터 정리 작업이 완료되었습니다.
4 . British Dominica (DM) 나라의 03. March, 나라별 Raw 데이터 정리 작업이 완료되었습니다.
5 . Brunei (BN) 나라의 03. March, 나라별 Raw 데이터 정리 작업이 완료되었습니다.
6 . Chile (CL) 나라의 03. March, 나라별 Raw 데이터 정리 작업이 완료되었습니다.
7 . Colombia (CO) 나라의 03. March, 나라별 Raw 데이터 정리 작업이 완료되었습니다.
8 . Costa Rica (CR) 나라의 03. March, 나라별 Raw 데이터 정리 작업이 완료되었습니다.
9 . Ecuador (EC) 나라의 03. March, 나라별 Raw 데이터 정리 작업이 완료되었습니다.
10 . Fiji (FJ) 나라의 03. March, 나라별 Raw 데이터 정리 작업이 완료되었습니다.
11 . Georgia (GE) 나라의 03. March, 나라별 Raw 데이터 정리 작업이 완료되었습니다.
12 . Honduras (HN) 나라의 03. March, 나라별 Raw 데이터 정리 작업이 완료되었습니다.
13 . Hong Kong (HK) 나라의 03. March, 나라별 Raw 데이터 정리 작업이 완료되었습니다.
14 . Indonesia (ID) 나라의 03. March, 나라별 Raw 데이터 정리 작업이 완료되었습니다.
15 . Kuwait (KW) 나라의 03. March, 나라별 Raw 데이터 정리 작업이 완료되었습니다.
16 . Laos (LA) 나라의 03. March, 나라별 Raw 데이터 정리 작업이 완료되었습니다.
17 . Lebanon (LB) 나라의 

## 모니터링 데이터 수집 관리 대장(월별), 각 파일 -> 관리대장

In [6]:
count_num = 0
row_i = 0

# 관리대장 파일 불러오기
final_workbook = openpyxl.load_workbook(final_address+"/"+"KIA 모니터링 데이터 수집 관리 대장_2020.xlsx")

final_2_backlilnks_sheet = final_workbook['2. Backlinks']
final_4_total_index_sheet = final_workbook['4. Total Indexed']
final_4_web_indexed_sheet = final_workbook['4-1. Submitted Web Indexed']
final_5_traffic_sheet = final_workbook['5. Organic Traffic']
final_6_device_sheet = final_workbook['6.Device']
final_7_bounce_sheet = final_workbook['7-1. Bounce Rate']
final_7_average_sheet = final_workbook['7-2. Average Time Spent']
final_8_kw_sheet = final_workbook['8-1. Organic KW']
final_8_kw_ranking_sheet = final_workbook['8-2. Organic KW Ranking']

for country, gather_country in zip(countrys, gather_countrys):
    
    # 가공 파일 불러오기
    gather_2_wb = openpyxl.load_workbook(global_address+"/"+country+"/"+month_address+"/"+
                                         "2. Off-page(Total Backlinks).xlsx")
    gather_2_ws = gather_2_wb['Off-page(Total Backlinks)']

    gather_2_1_wb = openpyxl.load_workbook(global_address+"/"+country+"/"+month_address+"/"+
                                           "2-1. Off-page(New Backlinks).xlsx")
    gather_2_1_ws = gather_2_1_wb['2-1. Off-page(New Backlinks)']

    gather_4_wb = openpyxl.load_workbook(global_address+"/"+country+"/"+month_address+"/"+
                                         "4. Indexing HTTPS.xlsx")
    gather_4_ws = gather_4_wb['Table']

    gather_5_wb = openpyxl.load_workbook(global_address+"/"+country+"/"+month_address+"/"+
                                         "5. Organic Traffic.xlsx")
    gather_5_ws = gather_5_wb['Organic Traffic']

    gather_6_wb = openpyxl.load_workbook(global_address+"/"+country+"/"+month_address+"/"+
                                         "6. Device Channel Traffic.xlsx")
    gather_6_ws = gather_6_wb['Worksheet']

    gather_7_wb = openpyxl.load_workbook(global_address+"/"+country+"/"+month_address+"/"+
                                         "7. Bounce Rate, Average Time Spent.xlsx")
    gather_7_ws = gather_7_wb['Worksheet']

    gather_8_wb = openpyxl.load_workbook(global_address+"/"+country+"/"+month_address+"/"+
                                         "8. Organic Keywords.xlsx")
    gather_8_ws = gather_8_wb['HTTPS']
    
    # 데이터 삽입
    row_i += 1
    
    if row_i == 7:
        row_i = 8
    elif row_i == 11:
        row_i = 12
    
    final_2_backlilnks_sheet['B'+column_address[gather_month*2-2]+
                             str(row_i+4)] = gather_2_1_ws.cell(row=2, column=1).value
    final_2_backlilnks_sheet['B'+column_address[gather_month*2-1]+
                             str(row_i+4)] = gather_2_ws.cell(row=2, column=1).value

    final_4_total_index_sheet['A'+column_address[gather_month+1]+
                              str(row_i+3)] =gather_4_ws.cell(row=1, column=8).value
    final_4_web_indexed_sheet['B'+column_address[gather_month*2-1]+
                                str(row_i+4)] = gather_4_ws.cell(row=2, column=8).value

    final_5_traffic_sheet['A'+column_address[gather_month+1]+
                          str(row_i+3)] = gather_5_ws.cell(row=3, column=7).value

    final_6_device_sheet['C'+column_address[gather_month*3-5]+
                         str(row_i+4)] = gather_6_ws.cell(row=19, column=6).value
    final_6_device_sheet['C'+column_address[gather_month*3-4]+
                         str(row_i+4)] = gather_6_ws.cell(row=19, column=7).value

    final_7_bounce_sheet['A'+column_address[gather_month+1]+
                         str(row_i+3)] = gather_7_ws.cell(row=17, column=4).value

    final_7_average_sheet['A'+column_address[gather_month+1]+
                          str(row_i+3)] = "{} AM".format(gather_7_ws.cell(row=17, column=3).value)

    final_8_kw_sheet['A'+column_address[gather_month+1]+
                     str(row_i+3)] = gather_8_ws.cell(row=2, column=2).value

    final_8_kw_ranking_sheet['A'+column_address[gather_month+1]+
                             str(row_i+3)] = gather_8_ws.cell(row=2, column=5).value

    # Raw 정리 파일 저장
    final_workbook.save(final_address+"/"+"KIA 모니터링 데이터 수집 관리 대장_2020_"+month_num+".xlsx")
    
    # 결과
    count_num += 1
    print(count_num,". {} 나라의 {}, 나라별 관리대장 작업이 완료되었습니다."
          .format(country, month_address))
    

1 . Aruba (AW) 나라의 03. March, 나라별 관리대장 작업이 완료되었습니다.
2 . Australia (AU) 나라의 03. March, 나라별 관리대장 작업이 완료되었습니다.
3 . Azerbaijan (AZ) 나라의 03. March, 나라별 관리대장 작업이 완료되었습니다.
4 . British Dominica (DM) 나라의 03. March, 나라별 관리대장 작업이 완료되었습니다.
5 . Brunei (BN) 나라의 03. March, 나라별 관리대장 작업이 완료되었습니다.
6 . Chile (CL) 나라의 03. March, 나라별 관리대장 작업이 완료되었습니다.
7 . Colombia (CO) 나라의 03. March, 나라별 관리대장 작업이 완료되었습니다.
8 . Costa Rica (CR) 나라의 03. March, 나라별 관리대장 작업이 완료되었습니다.
9 . Ecuador (EC) 나라의 03. March, 나라별 관리대장 작업이 완료되었습니다.
10 . Fiji (FJ) 나라의 03. March, 나라별 관리대장 작업이 완료되었습니다.
11 . Georgia (GE) 나라의 03. March, 나라별 관리대장 작업이 완료되었습니다.
12 . Honduras (HN) 나라의 03. March, 나라별 관리대장 작업이 완료되었습니다.
13 . Hong Kong (HK) 나라의 03. March, 나라별 관리대장 작업이 완료되었습니다.
14 . Indonesia (ID) 나라의 03. March, 나라별 관리대장 작업이 완료되었습니다.
15 . Kuwait (KW) 나라의 03. March, 나라별 관리대장 작업이 완료되었습니다.
16 . Laos (LA) 나라의 03. March, 나라별 관리대장 작업이 완료되었습니다.
17 . Lebanon (LB) 나라의 03. March, 나라별 관리대장 작업이 완료되었습니다.
18 . Malaysia (MY) 나라의 03. March, 나라별 관리대장 작업이 완료되었습니다.
19 . Ma

## 나라별 Raw 데이터 정리(반기별)
(반기별로 On_page_column과 response_column 숫자를 바꿔줘야 함)

In [ ]:
# count_num = 0

# for country, gather_country in zip(countrys, gather_countrys):
    
#     # 가공 파일 불러오기
#     gather_1_wb = openpyxl.load_workbook(global_address+"/"+country+"/"+month_address+"/"+
#                                      "1. On-page(Title, Description).xlsx")
#     gather_1_ws = gather_1_wb['On-Page SEO']
    
#     gather_3_wb = openpyxl.load_workbook(global_address+"/"+country+"/"+month_address+"/"+
#                                      "3. Response Codes.xlsx")
#     gather_3_ws = gather_3_wb['피벗']

#     # Raw 정리 파일 불러오기
#     gather_workbook = openpyxl.load_workbook(gather_address+"/"+gather_country+".xlsx")
    
#     gather_1_onpage_sheet = gather_workbook['On-Page']
#     gather_3_response_sheet = gather_workbook['Response']
    
#     # 데이터 삽입(반기별로 On_page_column과 response_column 숫자를 바꿔줘야 함)
#     On_page_column = 'AG'
#     gather_1_onpage_sheet[On_page_column+'4'] = gather_8_ws.cell(row=6, column=3).value
#     gather_1_onpage_sheet[On_page_column+'5'] = gather_8_ws.cell(row=7, column=3).value
#     gather_1_onpage_sheet[On_page_column+'6'] = gather_8_ws.cell(row=8, column=3).value
#     gather_1_onpage_sheet[On_page_column+'7'] = gather_8_ws.cell(row=9, column=3).value
#     gather_1_onpage_sheet[On_page_column+'8'] = gather_8_ws.cell(row=10, column=3).value
        
#     gather_1_onpage_sheet[On_page_column+'14'] = gather_8_ws.cell(row=14, column=3).value
#     gather_1_onpage_sheet[On_page_column+'15'] = gather_8_ws.cell(row=15, column=3).value
#     gather_1_onpage_sheet[On_page_column+'16'] = gather_8_ws.cell(row=16, column=3).value
#     gather_1_onpage_sheet[On_page_column+'17'] = gather_8_ws.cell(row=17, column=3).value
#     gather_1_onpage_sheet[On_page_column+'18'] = gather_8_ws.cell(row=18, column=3).value
    
#     response_column = 'I'
#     gather_3_response_sheet[response_column+'4'] = gather_3_ws.cell(row=2, column=3).value
#     gather_3_response_sheet[response_column+'5'] = gather_3_ws.cell(row=3, column=3).value
#     gather_3_response_sheet[response_column+'6'] = gather_3_ws.cell(row=4, column=3).value
#     gather_3_response_sheet[response_column+'7'] = gather_3_ws.cell(row=5, column=3).value
#     gather_3_response_sheet[response_column+'8'] = gather_3_ws.cell(row=6, column=3).value
    
#     # Raw 정리 파일 저장
#     gather_workbook.save(gather_address+"/"+gather_country+"_"+month_num+"(반기별).xlsx")
    
#     # 결과
#     count_num += 1
#     print(count_num,". {} 나라의 {}, 나라별 Raw 데이터 정리 작업이 완료되었습니다."
#           .format(country, month_address))

## 모니터링 데이터 수집 관리 대장(반기별), 각 파일 -> 관리대장
(반기별로 Response_sheet의 column명을 바꿔줘야 함)

In [ ]:
# count_num = 0
# row_i = 0

# # 관리대장 파일 불러오기
# final_workbook = openpyxl.load_workbook(final_address+"/"+
#                                         "KIA 모니터링 데이터 수집 관리 대장_2020.xlsx")

# final_1_title_sheet = final_workbook['1-1. On-page Title']
# final_1_description_sheet = final_workbook['1-1. On-page Description']
# final_3_response_sheet = final_workbook['3. Response Codes']

# for country, gather_country in zip(countrys, gather_countrys):
    
#     # 가공 파일 불러오기
#     gather_1_wb = openpyxl.load_workbook(global_address+"/"+country+"/"+month_address+"/"+
#                                      "1. On-page(Title, Description).xlsx")
#     gather_1_ws = gather_1_wb['On-Page SEO']
    
#     gather_3_wb = openpyxl.load_workbook(global_address+"/"+country+"/"+month_address+"/"+
#                                      "3. Response Codes.xlsx")
#     gather_3_ws = gather_3_wb['피벗']

#     # 데이터 삽입
#     row_i += 1
    
#     if row_i == 7:
#         row_i = 8
#     elif row_i == 11:
#         row_i = 12
    
#     final_1_title_sheet[column_address[gather_month*3-2]+str(row_i+4)] = gather_1_ws.cell(row=6, column=3).value
#     final_1_title_sheet[column_address[gather_month*3-1]+str(row_i+4)] = gather_1_ws.cell(row=7, column=3).value
    
#     final_1_description_sheet[column_address[gather_month*3-2]+
#                               str(row_i+4)] = gather_1_ws.cell(row=14, column=3).value
#     final_1_description_sheet[column_address[gather_month*3-1]+
#                               str(row_i+4)] = gather_1_ws.cell(row=15, column=3).value
    
#     final_3_response_sheet['BV'+str(row_i+6)] = gather_3_ws.cell(row=2, column=3).value
#     final_3_response_sheet['BW'+str(row_i+6)] = gather_3_ws.cell(row=3, column=3).value
#     final_3_response_sheet['BY'+str(row_i+6)] = gather_3_ws.cell(row=4, column=3).value
#     final_3_response_sheet['BZ'+str(row_i+6)] = gather_3_ws.cell(row=5, column=3).value
#     final_3_response_sheet['CA'+str(row_i+6)] = gather_3_ws.cell(row=6, column=3).value
    
#     # Raw 정리 파일 저장
#     final_workbook.save(final_address+"/"+"KIA 모니터링 데이터 수집 관리 대장_2020_"+
#                         month_num+"(반기별).xlsx")
    
#     # 결과
#     count_num += 1
#     print(count_num,". {} 나라의 {}, 나라별 Raw 데이터 정리 작업이 완료되었습니다."
#           .format(country, month_address))